### you can easily copy the following two cell to make it into a python file.
The features are listed:
#### sentiment:
1. vader sentiment score normal average for pos, neg, neu, compound   - 4 features
2. vader sentiment score average weighted on number of votes   - 4 features
3. vader sentiment score average weighted on number of next-level comments    - 4 features

we can also have these for socialsent.....

#### statistical:
1. number of comments in a day  - 1
2. number of child comments fo a comment in a day  - 1
3. avg length of text  - 1
4. proportion of types of user (bullish, ...) in a day  - 5

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

# some preprocess
# turn the vader dict columns into four seperate cols.
def vader_score(dataframe):
    text = pd.DataFrame({'vader_negative':[0]*dataframe.shape[0],'vader_neutral':[0]*dataframe.shape[0],
                         'vader_positive':[0] * dataframe.shape[0], 'vader_compound':[0]*dataframe.shape[0]})
    for i in range(dataframe.shape[0]):
        text['vader_negative'].iloc[i] = eval(data_demo['vader_scores'].iloc[i])['neg']
        text['vader_positive'].iloc[i] =  eval(data_demo['vader_scores'].iloc[i])['pos']
        text['vader_neutral'].iloc[i] =  eval(data_demo['vader_scores'].iloc[i])['neu']
        text['vader_compound'].iloc[i] =  eval(data_demo['vader_scores'].iloc[i])['compound']
    return text

def turn_datetime(dataframe):
    t = []
    for i in range(dataframe.shape[0]):
        Month = dataframe['daily_discussion_date'].iloc[i].split(',')[1].split(' ')[1][0:3]
        Day = dataframe['daily_discussion_date'].iloc[i].split(',')[1].split(' ')[2]
        year = dataframe['daily_discussion_date'].iloc[i].split(',')[2].split(' ')[-1]
        datetime_object = datetime.strptime(Month + ' ' + Day + ' ' + year, '%b %d %Y')
        t.append(datetime_object)
    dataframe['daily_discussion_date'] = pd.DataFrame(t)

# get sentiment features

# will do a demo of daily here. Will modify later to see whether we use hourly or 6-hour interval.
def sentiment_normal_avg_byday(df, lexicon_name):
    # lexicon name can only be "vader", "socialsent".
    sent_catogory = ["compound", "neutral", "negative", "positive"]
    col_names = [lexicon_name+"_"+i for i in sent_catogory]
    groupby_dict = {}
    for key in col_names:
        groupby_dict[key] = "mean"
    return data_demo.groupby("daily_discussion_date").agg(groupby_dict)

def sentiment_votes_avg_byday(df, lexicon_name):
    weighted_mean = lambda x: np.average(x, weights=df.loc[x.index, "upvotes"])
    # lexicon name can only be "vader", "socialsent".
    sent_catogory = ["compound", "neutral", "negative", "positive"]
    col_names = [lexicon_name+"_"+i for i in sent_catogory]
    renames = [lexicon_name+"_"+"votes"+"_"+i for i in sent_catogory]
    groupby_dict = {}
    for key in col_names:
        groupby_dict[key] = weighted_mean
    df_result = data_demo.groupby("daily_discussion_date").agg(groupby_dict)
    df_result.columns = renames
    return df_result

def sentiment_child_avg_byday(df, lexicon_name):
    weighted_mean = lambda x: np.average(x, weights=df.loc[x.index, "num_child"])
    # lexicon name can only be "vader", "socialsent".
    sent_catogory = ["compound", "neutral", "negative", "positive"]
    col_names = [lexicon_name+"_"+i for i in sent_catogory]
    renames = [lexicon_name+"_"+"child"+"_"+i for i in sent_catogory]
    groupby_dict = {}
    for key in col_names:
        groupby_dict[key] = weighted_mean
    df_result = data_demo.groupby("daily_discussion_date").agg(groupby_dict)
    df_result.columns = renames
    return df_result

# Bullish, Bearish, Long-term Holder, Bitcoin Skeptic, None
def get_author_opinion(dataframe):
    opinion_types = ['Bullish','Bearish','Long-term Holder','Bitcoin Skeptic','None']
    for type in opinion_types:
        dataframe[type] = 0
    for i in range(dataframe.shape[0]):
        dataframe.set_value(i, dataframe.iloc[i]["author_opinion"], 1)
    return dataframe

def opinion_avg_byday(df):
    opinion_types = ['Bullish','Bearish','Long-term Holder','Bitcoin Skeptic','None']
    groupby_dict = {}
    for type in opinion_types:
        groupby_dict[type] = "mean"
    return data_demo.groupby("daily_discussion_date").agg(groupby_dict)

In [4]:
%%time
# main function

# read the data you want
data_demo = pd.read_csv("./data/data/bitcoin_markets_daily_discussion_january_v1.csv")

data_demo = pd.merge(data_demo, vader_score(data_demo), left_index=True, right_index=True)
turn_datetime(data_demo)

# sentiment features:
print("extracting sentiment features....")
daily_feature = sentiment_normal_avg_byday(data_demo, "vader")
daily_feature = pd.concat([daily_feature, sentiment_votes_avg_byday(data_demo, "vader")], axis=1)
# get the number of next-layer comments
data_demo["num_child"] = data_demo["comment_id"].apply(lambda x: data_demo[data_demo["parent_id"]==x].shape[0])
daily_feature = pd.concat([daily_feature, sentiment_child_avg_byday(data_demo, "vader")], axis=1)

# statistical features:
print("extracting statistical features....")
data_demo = get_author_opinion(data_demo)

df_num_comments = data_demo.groupby("daily_discussion_date").size().to_frame()
df_num_comments.columns = ["num_comments"]

df_num_children = data_demo.groupby("daily_discussion_date").agg({"num_child":"mean"})

data_demo["num_word"] = data_demo["body"].apply(lambda x:len(x))
df_num_word = data_demo.groupby("daily_discussion_date").agg({"num_word":"mean"})

daily_feature = pd.concat([daily_feature, opinion_avg_byday(data_demo), 
                           df_num_comments, df_num_children, df_num_word], axis=1)

print("done! The data size is", data_demo.shape)

D:\Programming\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


extracting sentiment features....
extracting statistical features....
done! The data size is (46712, 20)
Wall time: 6min 39s


The following is the feature input we will use (Jan sample)

In [7]:
daily_feature

,vader_negative,vader_positive,vader_neutral,vader_compound,vader_votes_compound,vader_votes_neutral,vader_votes_negative,vader_votes_positive,vader_child_compound,vader_child_neutral,vader_child_negative,vader_child_positive,Bearish,None,Bullish,Bitcoin Skeptic,Long-term Holder,num_comments,num_child,num_word
daily_discussion_date,,,,,,,,,,,,,,,,,,,,
2018-01-01,0.071214,0.123046,0.805734,0.126620,0.057980,0.129303,0.812685,0.198863,0.074968,0.093485,0.831541,0.105663,0.042514,0.731978,0.083179,0.003697,0.138632,541,0.865065,213.743068
2018-01-02,0.065844,0.113365,0.819752,0.131937,0.060349,0.111737,0.827305,0.184096,0.063824,0.103987,0.832181,0.159920,0.034126,0.783868,0.057911,0.006205,0.117890,967,0.856256,197.833506
2018-01-03,0.075551,0.111242,0.813199,0.123640,0.083496,0.112270,0.804258,0.150481,0.073780,0.093607,0.832627,0.138089,0.015385,0.748416,0.070588,0.003620,0.161991,1105,0.875113,216.444344
2018-01-04,0.073283,0.112134,0.813771,0.083307,0.080932,0.107818,0.810787,0.098907,0.073298,0.091908,0.834811,0.068976,0.020425,0.781046,0.100490,0.004902,0.093137,1224,0.857026,178.022059
2018-01-05,0.069134,0.126343,0.804517,0.136669,0.068140,0.123352,0.808516,0.179115,0.070553,0.110307,0.819114,0.167191,0.016939,0.799247,0.082183,0.002509,0.099122,1594,0.859473,173.082183
2018-01-06,0.073268,0.127214,0.798341,0.145679,0.060172,0.138766,0.798569,0.211830,0.071205,0.106600,0.822167,0.142295,0.018540,0.697567,0.144844,0.002317,0.136732,863,0.866744,201.876014
2018-01-07,0.084094,0.123753,0.792146,0.097713,0.081457,0.120864,0.797649,0.129741,0.083595,0.105187,0.811207,0.088307,0.026620,0.810185,0.076389,0.000000,0.086806,864,0.842593,198.211806
2018-01-08,0.079094,0.123797,0.796422,0.095672,0.077610,0.122449,0.799486,0.104333,0.080078,0.099542,0.820383,0.056855,0.024641,0.759069,0.105407,0.002053,0.108830,1461,0.859685,180.431211
2018-01-09,0.079781,0.120771,0.798485,0.107711,0.079140,0.118187,0.801833,0.158298,0.079636,0.098067,0.822277,0.083838,0.045412,0.714286,0.126774,0.012299,0.101230,1057,0.855251,197.032167
